## [실습] Single Agent 구축부터 MCP(Model Context Protocol) 연동까지
> 학습 목표 :
> 1. **Handcrafted Agent**: **Prompt Engineering** 과 **Python** 만으로 에이전트의 사고 루프(Reasoning Loop)를 직접 구현합니다.
> 2. **Smolagents**: Hugging Face의 최신 라이브러리를 통해 **Code Agnet**의 강력함을 체험합니다.
> 3. **MCP Integration**: "LLM의 USB-C"라 불리는 MCP 서버를 직접 구축하고, 에이전트와 연동하여 도구를 확장합니다.

#### 0. 환경 설정 (Setup)
실습에 필요한 라이브러리
- smolagents: 가벼운 에이전트 프레임워크
- mcp, fastmcp: MCP 서버 및 클라이언트 구현체
- openai: LLM 백엔드 (또는 anthropic 등)

In [1]:
# 필요 패키지 설치
%pip install -q openai 'smolagents[litellm]' mcp fastmcp python-dotenv nest_asyncio ddgs

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import nest_asyncio
from dotenv import load_dotenv

# Jupyter에서 비동기 루프 실행을 위한 설정 (MCP 통신용)
nest_asyncio.apply()

# API Key 설정(OpenAI 권장)
load_dotenv()

True

---

#### 1. Handcrafted Agent: ReAct 패턴의 해부
> **이론 배경**:
>  
> ReAct(Reasoning + Acting)는 **Thought(사고) → Action(행동) → Observation(관찰)**의 루프를 돕니다. 
> 프레임워크 없이 이를 구현해 보면, 에이전트가 어떻게 **Stop Sequence**를 인식하고 도구를 호출하는지 정확히 알 수 있습니다.
##### 1-1. 도구(Tools) 및 프롬프트 정의
에이전트에게 "시간 확인"과 "계산" 능력을 부여합니다.

In [3]:
import datetime
import json
from zoneinfo import ZoneInfo


# 1. 도구 정의 (실제 함수)
def get_current_time(timezone: str = "Asia/Seoul") -> str:
    """현재 시간을 반환합니다. timezone 예: 'UTC', 'Asia/Seoul'"""
    try:
        # [FIX] ZoneInfo를 사용하여 실제 timezone을 적용
        now = datetime.datetime.now(ZoneInfo(timezone))
        return now.strftime(f"%Y-%m-%d %H:%M:%S ({timezone})")
    except Exception:
        return "Invalid Timezone"


def calculator(expression: str) -> str:
    """수식을 문자열로 받아 계산합니다. 예: '2 + 2 * 3'"""
    try:
        # 보안상 eval은 위험하지만 예제 단수화를 위해 사용
        return str(eval(expression, {"__builtins__": None}, {}))
    except Exception as e:
        return f"Error: {e}"


# 도구 매핑 테이블
tools_map = {"get_current_time": get_current_time, "calculator": calculator}

# 2. System Prompt (ReAct Style)
REACT_SYSTEM_PROMPT = """
You are a helpful AI assistant.
To answer the user's question, you function in a loop of Thought, Action, Observation.

AVAILABLE TOOLS:
- get_current_time(timezone): Get current time. Example args: {"timezone": "Asia/Seoul"}
- calculator(expression): Calculate a math expression. Example args: {"expression": "2 * 3 + 1"}

FORMAT INSTRUCTIONS:
Use the following format strictly:

Question: the input question you must answer
Thought: you should always think about what to do
Action:
```json
{
  "tool_name": "tool_name_here",
  "arguments": {"param_name": "value"}
}
```
Observation: the result of the action (provided by system)
... (this Thought/Action/Observation can repeat N times)
Final Answer: the final answer to the original input question

IMPORTANT: The "arguments" field must always be a JSON object (dict), not a string.

Begin!
"""

##### 1-2. ReAct Agent Class 구현 (The Core Loop)
LLM의 응답을 파싱하고, 도구를 실행한 뒤, 그 결과를 다시 프롬프트에 붙이서 LLM에게 던져주는 **핵심 로직**입니다.

In [4]:
import re
from openai import OpenAI


class HandcraftedAgent:
    def __init__(self, system_prompt, tools_map, model="gpt-4.1"):
        self.client = OpenAI()
        self.system_prompt = system_prompt
        self.tools_map = tools_map
        self.model = model
        self.max_steps = 5

    def chat(self, user_input):
        # 대화 기록(Context) 초기화
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": f"Question: {user_input}"},
        ]

        print(f"[Start Task]: {user_input}")

        for step in range(self.max_steps):
            # 1. LLM 호출
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                stop=["Observation:"],  # LLM이 스스로 관찰 결과를 지어내지 못하게 멈춤
            )
            llm_output = response.choices[0].message.content
            print(f"\n--- Step {step +1} ---\n{llm_output}")

            # 메시지 기록에 LLM의 생각 추가
            messages.append({"role": "assistant", "content": llm_output})

            # 2. 종료 조건 확인
            if "Final Answer:" in llm_output:
                return llm_output.split("Final Answer:")[-1].strip()

            # 3. Action 파싱 (JSON 추출)
            # Markdown 코드 블록 ```json ...``` 사이의 내용을 추출
            json_match = re.search(r"```json\s*(\{.*?\})\s*```", llm_output, re.DOTALL)

            if json_match:
                try:
                    action_data = json.loads(json_match.group(1))
                    tool_name = action_data.get("tool_name")
                    args = action_data.get("arguments")

                    # 4. 도구 실행 (Execution)
                    print(f"[Tool Call]: {tool_name} with args {args}")
                    if tool_name in self.tools_map:
                        # [FIX] args가 dict이면 **kwargs로 전달, 문자열이면 그대로 전달
                        if isinstance(args, dict):
                            observation = self.tools_map[tool_name](**args)
                        else:
                            observation = self.tools_map[tool_name](args)
                    else:
                        observation = f"Error: Tool '{tool_name}' not found."

                except Exception as e:
                    observation = f"Error parsing/execution acton: {str(e)}"
            else:
                observation = (
                    "Error: Invalid Action Format. Please provide Action in JSON block."
                )

            # 5. 관잘 결과(Observation)를 Context에 주입
            print(f"[Observation]: {observation}")

            # 다음 턴을 위해 User 메시지로 관찰 결과 전달
            messages.append({"role": "user", "content": f"Observation: {observation}"})

        return "Max steps reached without final answer."


# --- 실행 테스트 ---
agent = HandcraftedAgent(REACT_SYSTEM_PROMPT, tools_map)
result = agent.chat(
    "지금 서울 시간을 확인하고, 해당 연도에 2025를 더하면 몇인지 계산해줘."
)
print(f"\n[Final Result]: {result}")

[Start Task]: 지금 서울 시간을 확인하고, 해당 연도에 2025를 더하면 몇인지 계산해줘.

--- Step 1 ---
Thought: 먼저, 현재 서울 시간을 확인하여 연도를 알아야 한다. 그 다음, 해당 연도에 2025를 더하여 결과를 계산한다.
Action:
```json
{
  "tool_name": "get_current_time",
  "arguments": {"timezone": "Asia/Seoul"}
}
```

[Tool Call]: get_current_time with args {'timezone': 'Asia/Seoul'}
[Observation]: 2026-02-13 19:18:19 (Asia/Seoul)

--- Step 2 ---
Thought: 현재 연도는 2026년이다. 이제 2026에 2025를 더하는 계산을 해야 한다.
Action:
```json
{
  "tool_name": "calculator",
  "arguments": {"expression": "2026 + 2025"}
}
```

[Tool Call]: calculator with args {'expression': '2026 + 2025'}
[Observation]: 4051

--- Step 3 ---
Final Answer: 지금 서울 시간의 연도는 2026년이고, 여기에 2025를 더하면 4051입니다.

[Final Result]: 지금 서울 시간의 연도는 2026년이고, 여기에 2025를 더하면 4051입니다.


---

#### 2. Using Framework: smolagents (Code Agent)
> 이론 배경: smolagents는 LLM이 JSON이 아닌 **Python Code**를 직접 생성하여 도구를 호출하는 **Code Agent** 방식을 사용합니다. 이는 복잡한 논리(Loop, if문)을 표현하는 데 JSON보다 훨씬 효율적입니다.

##### 2-1. Python 함수 기반 도구 생성
@tool 데코데이터를 사용하여 함수를 도구로 변환합니다. Docstring과 Type Hint가 매우 중요합니다.

In [6]:
from smolagents import CodeAgent, Tool, LiteLLMModel, tool


# 기존 함수를 smolagents용 도구로 래핑
@tool
def check_server_status(server_name: str) -> str:
    """
    Checks the health status of a specific server.

    Args:
        server_name: The name of the server to check (e.g., 'prod-db', 'web-01').
    """

    # 더미 로직
    if "prod" in server_name:
        return f"[ALERT] {server_name} is experiencing high load (CPU 95%)."
    return f"{server_name} is running smoothly."


@tool
def restart_server(server_name: str) -> str:
    """
    Restarts the specified server. Use this only when the server status is bad.

    Args:
        server_name: The name of the server to restart.
    """

    return f"Success: {server_name} has been restarted and is now healthy."

##### 2-2. Code Agent 실행
CodeAgent 는 내부적으로 **Python Interpreter**를 가지고 있어, LLM이 짠 코드를 샌드박스 환경에서 안전하게 실행합니다.

In [7]:
# 모델 설정 (OpenAI gpt-4o 사용)
model = LiteLLMModel(model_id="gpt-4.1")

# 에이전트 생성
manager_agent = CodeAgent(
    tools=[check_server_status, restart_server],  # 우리가 만든 도구들
    model=model,
    add_base_tools=True,
    max_steps=6,
)

print("--- smolagents: Server Manager Scenario ---")
# 시나리오: 상태 확인 후 문제가 있으면 재시작까지 수행하는 복합 작업
response = manager_agent.run(
    "Check the status of 'prod-db'. If there is a problem, fix it and confirm the status again."
)

print(f"\nAgent Answer: {response}")

--- smolagents: Server Manager Scenario ---


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Check the status of 'prod-db'. If there is a problem, fix it and confirm the status again.                      │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1 ────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  status = check_server_status("prod-db")                                                                          
  print(f"Initial status: {status}")                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Initial status: [ALERT] prod-db is experiencing high load (CPU 95%).

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Checked server 'prod-db'; initial issue: high load (CPU 95%). Restarted the server, but the        
  problem persists (CPU 95% load). Please investigate further, as restart did not resolve the issue.")             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Checked server 'prod-db'; initial issue: high load (CPU 95%). Restarted the server, but the problem 
persists (CPU 95% load). Please investigate further, as restart did not resolve the issue.

[Step 3: Duration 3.51 seconds| Input tokens: 7,157 | Output tokens: 302]


Agent Answer: Checked server 'prod-db'; initial issue: high load (CPU 95%). Restarted the server, but the problem persists (CPU 95% load). Please investigate further, as restart did not resolve the issue.


---

#### 3. Tool Access: MCP (Model Context Protocol) 연동
> 이론 배경:
> **MCP**는 LLM과 도구 사이의 표준 프로토콜입니다.
> - 기존: 모든 앱(LangChain, Semantic Kernel 등) 마다 도구 연결 코드를 따로 짜야 함.
> - MCP: 서버 하나만 만들어두면 Claude Desktop, Cursor, 그리고 우리가 만든 Agent 어디서든 연결 가능 (Plug-and-Play).

##### 3-1. MCP Server 구축 (file: myserver.py)
fastmcp를 사용하여 별도의 프로세스로 동작하는 MCP 서버를 만듭니다. 주피터 환경 한계를 극복하기 위해  %%writefile 매직 커맨드로 파일을 생성합니다.

In [ ]:
%%writefile myserver.py
import json
from fastmcp import FastMCP

# 1. MCP 서버 인스턴스 생성
mcp = FastMCP("Advanced Data Tools")

# 2. 도구 등록
@mcp.tool()
def analyze_log_file(filepath: str) -> str:
    """
    Analyzes a log file and returns a summary of errors.
    Mock function for demo.
    """
    return f"Checked {filepath}: Found 3 Critical Errors, 12 Warnings."

@mcp.tool()
def query_customer_db(customer_id: str) -> str:
    """
    Retrieves customer information from the secure database.
    Returns a JSON string of the customer record.
    """
    # [FIX] dict 대신 JSON 문자열을 반환하여 MCP text 응답과 호환
    if customer_id == "C123":
        return json.dumps({"name": "Kim Agent", "plan": "Premium", "last_login": "2024-02-07"})
    return json.dumps({"error": "Customer not found"})

# 3. 서버 실행
if __name__ == "__main__":
    mcp.run()

##### 3-2. MCP Client Adapter 구현 (Crucial Step)
smolagents는 기본적으로 MCP를 직접 지원하지 않으므로, **MCP Client → Python function**으로 변환해주는 어댑터(Adapter)를 구현해야 합니다.

In [ ]:
import subprocess
import sys
import json
from smolagents import Tool

# 별도 프로세스에서 실행될 MCP 헬퍼 스크립트
_MCP_HELPER = """
import asyncio, json, sys
from mcp.client.sse import sse_client
from mcp import ClientSession

async def main():
    req = json.loads(sys.stdin.read())
    async with sse_client(req["url"] + "/sse") as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            if req["action"] == "list_tools":
                result = await session.list_tools()
                out = []
                for t in result.tools:
                    inputs = {}
                    if t.inputSchema and "properties" in t.inputSchema:
                        for pname, pschema in t.inputSchema["properties"].items():
                            inputs[pname] = {
                                "type": pschema.get("type", "string"),
                                "description": pschema.get("description", ""),
                            }
                    out.append({"name": t.name, "description": t.description or "No description.", "inputs": inputs})
                print(json.dumps(out))
            elif req["action"] == "call_tool":
                result = await session.call_tool(req["tool_name"], arguments=req.get("arguments", {}))
                text = result.content[0].text if result.content else "No content returned."
                print(json.dumps({"text": text}))

asyncio.run(main())
"""


class MCPAdapterSSE:
    """HTTP/SSE 방식으로 MCP 서버와 통신하는 어댑터 (subprocess 기반, Jupyter 완벽 호환)"""

    def __init__(self, base_url="http://localhost:8000"):
        self.base_url = base_url.rstrip("/")

    def _mcp_call(self, action, **kwargs):
        payload = json.dumps({"url": self.base_url, "action": action, **kwargs})
        result = subprocess.run(
            [sys.executable, "-c", _MCP_HELPER],
            input=payload,
            capture_output=True,
            text=True,
            timeout=30,
        )
        if result.returncode != 0:
            raise RuntimeError(f"MCP Error: {result.stderr.strip()}")
        return json.loads(result.stdout)

    def get_smolagents_tools(self):
        tool_defs = self._mcp_call("list_tools")
        return [self._create_wrapper(td) for td in tool_defs]

    def _create_wrapper(self, td):
        adapter = self
        _name = td["name"]
        _desc = td["description"]
        _inputs = td["inputs"]
        param_names = list(_inputs.keys())

        # forward 메서드의 시그니처를 inputs 키와 일치시켜야 함
        params_str = ", ".join(param_names)
        kwargs_str = ", ".join(f'"{p}": {p}' for p in param_names)
        exec_ns = {"_adapter": adapter, "_tool_name": _name}
        exec(
            f"def forward(self, {params_str}):\n"
            f"    result = _adapter._mcp_call('call_tool', tool_name=_tool_name, arguments={{{kwargs_str}}})\n"
            f"    return result['text']",
            exec_ns,
        )

        tool_cls = type(
            _name,
            (Tool,),
            {
                "name": _name,
                "description": _desc,
                "inputs": _inputs,
                "output_type": "string",
                "forward": exec_ns["forward"],
            },
        )
        return tool_cls()

##### 3-3. Final Integration: Agent + MCP
이제 직접 만든 myserver.py 를 Agent에 연결합니다. 마치 USB 장치를 꽂듯이 도구가 확장되는 것을 확인하세요.

In [9]:
# 먼저 별도 터미널에서 MCP 서버를 실행하세요.
# fastmcp run myserver.py --transport sse --port 8000

# 1. SSE 어탭터로 연결
print(" Connecting to MCP Server (SSE)...")
adapter = MCPAdapterSSE("http://localhost:8000")

# 2. 도구 로드
mcp_tools = adapter.get_smolagents_tools()
print(f"Loaded Tools from MCP: {[t.name for t in mcp_tools]}")

# 3. Agent 생성
final_agent = CodeAgent(tools=mcp_tools, model=model, add_base_tools=True)

print("\n[Scenario]: Customer Support with MCP Tools")
prompt = """
Customer ID 'C123' has reported an issue.
1. Retrieve the customer's profile using the database tool.
2. Analyze the system log file named 'system_error.log'.
3. Provide a brief report combining the customer info and log analysis.
"""

result = final_agent.run(prompt)
print(f"\nFinal Report:\n{result}")

 Connecting to MCP Server (SSE)...
Loaded Tools from MCP: ['analyze_log_file', 'query_customer_db']

[Scenario]: Customer Support with MCP Tools


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Customer ID 'C123' has reported an issue.                                                                       │
│ 1. Retrieve the customer's profile using the database tool.                                                     │
│ 2. Analyze the system log file named 'system_error.log'.                                                        │
│ 3. Provide a brief report combining the customer info and log analysis.                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1 ────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  customer_info = query_customer_db(customer_id='C123')                                                            
  print(f"Customer Info: {customer_info}")                                                                         
  log_summary = analyze_log_file(filepath='system_error.log')                                                      
  print(f"Log Analysis: {log_summary}")                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Customer Info: {"name": "Kim Agent", "plan": "Premium", "last_login": "2024-02-07"}
Log Analysis: Checked system_error.log: Found 3 Critical Errors, 12 Warnings.

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  report = (                                                                                                       
      "Customer Issue Report:\n"                                                                                   
      "----------------------\n"                                                                                   
      f"Name: Kim Agent\n"                                                                                         
      f"Plan: Premium\n"                                                                                           
      f"Last Login: 2024-02-07\n\n"                                                                                
      "System Log Analysis:\n"                                                                                     
      "Checked system_error.log: Found 3 Critical Errors, 12 Warnings."                                            
  )                                                                                                                
  final_answer(report)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Customer Issue Report:
----------------------
Name: Kim Agent
Plan: Premium
Last Login: 2024-02-07

System Log Analysis:
Checked system_error.log: Found 3 Critical Errors, 12 Warnings.

[Step 2: Duration 3.28 seconds| Input tokens: 4,675 | Output tokens: 239]


Final Report:
Customer Issue Report:
----------------------
Name: Kim Agent
Plan: Premium
Last Login: 2024-02-07

System Log Analysis:
Checked system_error.log: Found 3 Critical Errors, 12 Warnings.


#### 4. Summary & Next Steps
오늘 실습을 통해 우리는 핵심 개념들을 모두 코드로 구현했습니다.
1. Handcrafted Agent: Stop Sequence와 JSON Parsing을 이용해 ReAct Loop을 직접 제어했습니다.
2. Smolagents: Python 코드를 생성하여 논리적인 작업수행이 가능한 Code Agent를 경헙했습니다.
3. MCP: 로컬 서버(myserver.py)를 띄우고, 이를 표준 프로토콜로 연결하여 확장성 있는 아키텍쳐를 구혔했습니다.

---

## [과제]
#### MCP Marketplace에서 원하는 MCP 찾아 agent에 연동하기

##### 1. Remote MCP Server 호출

In [ ]:
# MCP: DuckDuckGo & Felo AI Search(덕덕고 웹 검색도구)를 호출하여 실시간 웹검색 결과를 조회한다.
# smithery MCP 사이트: https://smithery.ai/servers/OEvortex/ddg_search
# Remote 호출방식

# 1. DuckDuckGo MCP Tool 정의
# MCP 서버의 기능을 에이전트가 사용할 수 있는 'Tool' 형태로 래핑합니다[5].
# Smithery의 OEvortex/ddg_search 명세를 참고하여 입력과 출력을 정의합니다.
from ddgs import DDGS


class DuckDuckGoSearchTool(Tool):
    name = "duckduckgo_search"
    description = """
    Performs a web search using DuckDuckGo. 
    Use this tool to find real-time information, news, or answers to questions 
    that require external knowledge not present in the model's training data.
    Returns a list of search results with titles, links, and snippets.
    """
    inputs = {
        "query": {
            "type": "string",
            "description": "The search query string. Be specific for better results.",
        },
        "max_results": {
            "type": "integer",
            "description": "The maximum number of results to return (default: 3).",
            "nullable": True,
        },
    }
    output_type = "string"

    def forward(self, query: str, max_results: int = 3) -> str:
        """
        Executes the search using DuckDuckGo (Simulating the MCP Server action)
        """
        try:
            results = []
            # DDGS 라이브러리를 사용하여 실제 검색 수행
            with DDGS() as ddgs:
                search_gen = ddgs.text(query, max_results=max_results)
                for r in search_gen:
                    results.append(
                        f"Title: {r['title']}\nLink: {r['href']}\nSnippet: {r['body']}\n"
                    )

            if not results:
                return "No results found."

            return "\n".join(results)
        except Exception as e:
            return f"Error during DuckDuckGo search: {str(e)}"


# 2. 모델 설정
# model = LiteLLMModel(model_id="gpt-4.1")

# 3. 에이전트 생성 및 툴 등록
# 생성한 DuckDuckGo 툴을 리스트에 담아 에이전트에게 쥐어줍니다[6].
agent = CodeAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model,
    add_base_tools=True,  # 기본적인 계산, 방문 도구 등 포함
)

# 4. 에이전트 실행 (검증)
# 최신 정보가 필요한 질문을 던져 MCP 도구를 잘 사용하는지 확인합니다.
print(">>> Agent is searching with DuckDuckGo...")
question = "삼성전자와 SK하이닉스에서 생산되는 대표 메모리의 종류를 기반으로 향후 주가예측을 하려고해. DuckDuckGo로 필요한 자료를 검색해서 주가의 추이를 예측해줘."

response = agent.run(question)

print("\n>>> Final Answer:")
print(response)

>>> Agent is searching with DuckDuckGo...


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 삼성전자와 SK하이닉스에서 생산되는 대표 메모리의 종류를 기반으로 향후 주가예측을 하려고해. DuckDuckGo로 필요한  │
│ 자료를 검색해서 주가의 추이를 예측해줘.                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1 ────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # 삼성전자, SK하이닉스의 대표 메모리 종류 검색                                                                   
  samsung_memory = duckduckgo_search("삼성전자 대표 메모리 종류", 3)                                               
  print("삼성전자 대표 메모리:", samsung_memory)                                                                   
                                                                                                                   
  skhynix_memory = duckduckgo_search("SK하이닉스 대표 메모리 종류", 3)                                             
  print("SK하이닉스 대표 메모리:", skhynix_memory)                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
삼성전자 대표 메모리: Title: 메모리사업부 | 사업영역 | 삼성반도체
Link: https://semiconductor.samsung.com/kr/about-us/business-area/memory/
Snippet: 삼성반도체 메모리 사업부는 휘발성 메모리인 DRAM, 비휘발성 메모리인 NAND 등 최고 성능과 최첨단 기술력으로 
세계 메모리 반도체 분야에서 선두를 유지하고 있습니다.

Title: 2026 삼성 메모리 전체 라인업 제품명 총정리
Link: https://tali.kr/samsung-memory
Snippet: Jan 1, 2026 · 도대체 뭐가 뭔지 모르겠다는 반응이 대부분이다. 이번 포스팅에서는 2026년 기준 삼성 메모리 
제품을 5개 카테고리로 나눠 최근 2-3년 출시 제품 위주로 정리하고 용도별 추천까지 담았다.

Title: 삼성전자, HBM4 세계 최초 출하···메모리 초격차 재시동 - 뉴스웨이
Link: https://www.newsway.co.kr/news/view?ud=2026021215161580624
Snippet: 1 day ago · 삼성 전자가 세계 최초로 HBM4 고대역폭 메모리 양산을 시작했다. 1c D램과 4나노 파운드리 공정 등 
첨단 기술을 적용해 성능, 전력 효율, 대역폭 모두 대폭 개선했다. 글로벌 AI, 데이터센터 시장을 겨냥해 생산라인을 
확장하며, 하반기 HBM4E 샘플 출하도 준비 중이다.

SK하이닉스 대표 메모리: Title: SK하이닉스 주요 제품군 심층 분석 (HBM3E, DDR5, 낸드플래시)
Link: https://kimanlife.com/entry/SK하이닉스-주요-제품군-심층-분석-HBM3E-DDR5-낸드플래시
Snippet: Apr 29, 2025 · SK 하이 닉스 RAM 이미지 디스크립션 SK 하이 닉스 는 글로벌 메모리 반도체 시장을 주도하는 
핵심 기업입니다. 특히 2024년 기준, 고대역폭 메모리 (HBM3E), 차세대 D램 (DDR5), 그리고 고성능 낸드플래시 기술에서 
세계적인 경쟁력을 확보하고 있어요.

Title: SK하이닉스 간단한 회사 소개 : 2023.8.14 기준 - 메모리반도체, DRAM...
Link: https://blog.naver.com/beer50cc/223210996804
Snippet: 휘발성 메모리 제품은 전원이 끊어지면 정보가 지워지는 반면, 비휘발성 제품은 전원이 끊겨도 저장된 정보가 
계속 남아 있습니다. 당사는 휘발성 메모리 인 DRAM과 비휘발성 메모리 인 NAND Flash를 주력 생산하고 있습니다.

Title: SK하이닉스/기술 - 나무위키
Link: https://namu.wiki/w/SK하이닉스/기술
Snippet: Oct 30, 2024 · 2.2. 제품 2.2.1. DDR3 SDRAM 2.2.2. DDR4 SDRAM 2.2.3. DDR5 SDRAM 2.2.4. LPDDR4 SDRAM 2.2.5. 
LPDDR5 SDRAM 2.2.6. HBM 3. 플래시 메모리 3.1. 제조 공정 3.1.1. 36단 3.1.2. 48단 3.1.3. 72단 3.1.4. 96단 3.1.5. 
128단 3.1.6. 176단 3.1.7. 238단 3.1.8. 321단


Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  summary = """                                                                                                    
  - 삼성전자와 SK하이닉스 모두 DRAM(특히 DDR5), NAND, HBM 등 첨단 메모리 반도체가 매출과 주가의 핵심 동력입니다.   
  - 최근 AI붐, 데이터센터 증설에 따른 HBM·DDR5 등 고성능 메모리 수요가 급증하면서 두 기업 모두 실적이 빠르게 개선  
  중입니다.                                                                                                        
  - 2026년까지 HBM4, HBM3E 등 차세대 고대역폭 메모리와 DDR5 등에서 '수퍼사이클'이 이어질 전망이며, 시장에서의      
  점유율 및 경쟁력에 따라 각각 삼성전자와 SK하이닉스 모두 공격적인 증설과 투자로 호황이 지속될 것으로 예측되고     
  있습니다.                                                                                                        
  - SK하이닉스 주가는 2025~2026년 4~5배 상승세를 경험했고, 삼성전자 역시 DRAM 가격 상승 및 AI 반도체 매출 확대를   
  발판 삼아 목표주가 15만원(실제 시총 1,000조원 이상)도 언급되고 있습니다.                                         
  - 단, 해당 전망은 AI 반도체 수요 둔화, 메모리 가격 변동, 미중 기술 분쟁 등 외부 변수에 민감하므로 변동성은       
  상존합니다.                                                                                                      
                                                                                                                   
  결론적으로, 주가 추이는 AI 시장 성장 및 HBM/DDR5 수요에 따라 2026년까지는 우상향(상승 추세)이 유력하나, 일부     
  단기 변동성에도 유의해야 합니다.                                                                                 
  """                                                                                                              
                                                                                                                   
  final_answer(summary)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 
- 삼성전자와 SK하이닉스 모두 DRAM(특히 DDR5), NAND, HBM 등 첨단 메모리 반도체가 매출과 주가의 핵심 동력입니다.
- 최근 AI붐, 데이터센터 증설에 따른 HBM·DDR5 등 고성능 메모리 수요가 급증하면서 두 기업 모두 실적이 빠르게 개선 
중입니다.
- 2026년까지 HBM4, HBM3E 등 차세대 고대역폭 메모리와 DDR5 등에서 '수퍼사이클'이 이어질 전망이며, 시장에서의 점유율 
및 경쟁력에 따라 각각 삼성전자와 SK하이닉스 모두 공격적인 증설과 투자로 호황이 지속될 것으로 예측되고 있습니다.
- SK하이닉스 주가는 2025~2026년 4~5배 상승세를 경험했고, 삼성전자 역시 DRAM 가격 상승 및 AI 반도체 매출 확대를 발판
삼아 목표주가 15만원(실제 시총 1,000조원 이상)도 언급되고 있습니다.
- 단, 해당 전망은 AI 반도체 수요 둔화, 메모리 가격 변동, 미중 기술 분쟁 등 외부 변수에 민감하므로 변동성은 
상존합니다.

결론적으로, 주가 추이는 AI 시장 성장 및 HBM/DDR5 수요에 따라 2026년까지는 우상향(상승 추세)이 유력하나, 일부 단기 
변동성에도 유의해야 합니다.

[Step 3: Duration 12.54 seconds| Input tokens: 10,512 | Output tokens: 1,061]


>>> Final Answer:

- 삼성전자와 SK하이닉스 모두 DRAM(특히 DDR5), NAND, HBM 등 첨단 메모리 반도체가 매출과 주가의 핵심 동력입니다.
- 최근 AI붐, 데이터센터 증설에 따른 HBM·DDR5 등 고성능 메모리 수요가 급증하면서 두 기업 모두 실적이 빠르게 개선 중입니다.
- 2026년까지 HBM4, HBM3E 등 차세대 고대역폭 메모리와 DDR5 등에서 '수퍼사이클'이 이어질 전망이며, 시장에서의 점유율 및 경쟁력에 따라 각각 삼성전자와 SK하이닉스 모두 공격적인 증설과 투자로 호황이 지속될 것으로 예측되고 있습니다.
- SK하이닉스 주가는 2025~2026년 4~5배 상승세를 경험했고, 삼성전자 역시 DRAM 가격 상승 및 AI 반도체 매출 확대를 발판 삼아 목표주가 15만원(실제 시총 1,000조원 이상)도 언급되고 있습니다.
- 단, 해당 전망은 AI 반도체 수요 둔화, 메모리 가격 변동, 미중 기술 분쟁 등 외부 변수에 민감하므로 변동성은 상존합니다.

결론적으로, 주가 추이는 AI 시장 성장 및 HBM/DDR5 수요에 따라 2026년까지는 우상향(상승 추세)이 유력하나, 일부 단기 변동성에도 유의해야 합니다.



##### 2. Local MCP 호출

##### 2-1. fastmcp를 사용하여 구동할 서버 파일 생성

In [ ]:
%%writefile ddgs_server.py
from fastmcp import FastMCP
from ddgs import DDGS

# 1. MCP 서버 정의
mcp = FastMCP("ddg_search_server")

# 2. DuckDuckGo 검색 도구 등록
@mcp.tool()
def duckduckgo_search(query: str, max_results: int = 3) -> str:
    """
    Performs a web search using DuckDuckGo.
    Use this to find real-time information.
    """
    try:
        results = []
        with DDGS() as ddgs:
            search_gen = ddgs.text(query, max_results=max_results)
            for r in search_gen:
                results.append(f"Title: {r['title']}\nLink: {r['href']}\nSnippet: {r['body']}\n")
        return "\n".join(results) if results else "No results found."
    except Exception as e:
        return f"Error: {str(e)}"

# 3. 서버 실행 (SSE 방식, 포트 8000)
if __name__ == "__main__":
    print(">>> Starting DuckDuckGo MCP Server on port 8000...")
    mcp.run(transport="sse", port=8000)

##### 2-1. Local MCP Server 실행 방식

In [11]:
# MCP: DuckDuckGo & Felo AI Search(덕덕고 웹 검색도구)를 호출하여 실시간 웹검색 결과를 조회한다.
# smithery MCP 사이트: https://smithery.ai/servers/OEvortex/ddg_search
# 먼저 별도 터미널에서 MCP 서버를 실행하세요.
# fastmcp run ddgs_server.py --transport sse --port 8000

# 1. SSE 어탭터로 연결
print(" Connecting to DuckDuckGo Search MCP Server (SSE)...")
adapter = MCPAdapterSSE("http://localhost:8000")

# 2. 도구 로드
mcp_tools = adapter.get_smolagents_tools()
print(f"Loaded Tools from MCP: {[t.name for t in mcp_tools]}")

# 3. Agent 생성
final_agent = CodeAgent(tools=mcp_tools, model=model, add_base_tools=True)

print("\n[Scenario]: 삼성전자와 SK하이닉스 가격 분석")
# question = "삼성전자와 SK하이닉스에서 생산되는 대표 메모리의 종류를 기반으로 향후 주가예측을 하려고해. DuckDuckGo로 필요한 자료를 검색해서 주가의 추이를 예측해줘."
prompt = """
I need to forecast the future stock price trends of Samsung Electronics and SK Hynix based on their representative memory products.

1. Use the 'duckduckgo_search' tool to find the latest market trends and outlooks for key memory types produced by both companies, specifically focusing on HBM (High Bandwidth Memory), DDR5, and NAND Flash.
2. Search for recent analyst reports or news articles regarding the revenue forecasts and stock price predictions for Samsung Electronics and SK Hynix for 2024-2025, linked to their AI memory chip performance.
3. Synthesize the search results to provide a comparative analysis of their competitive strengths and a brief prediction of their stock price trends.
4. Answer in Korean.
"""

result = final_agent.run(prompt)
print(f"\nFinal Report:\n{result}")

 Connecting to DuckDuckGo Search MCP Server (SSE)...
Loaded Tools from MCP: ['duckduckgo_search']

[Scenario]: 삼성전자와 SK하이닉스 가격 분석


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I need to forecast the future stock price trends of Samsung Electronics and SK Hynix based on their             │
│ representative memory products.                                                                                 │
│                                                                                                                 │
│ 1. Use the 'duckduckgo_search' tool to find the latest market trends and outlooks for key memory types produced │
│ by both companies, specifically focusing on HBM (High Bandwidth Memory), DDR5, and NAND Flash.                  │
│ 2. Search for recent analyst reports or news articles regarding the revenue forecasts and stock price           │
│ predictions for Samsung Electronics and SK Hynix for 2024-2025, linked to their AI memory chip performance.     │
│ 3. Synthesize the search results to provide a comparative analysis of their competitive strengths and a brief   │
│ prediction of their stock price trends.                                                                         │
│ 4. Answer in Korean.                                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1 ────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: HBM, DDR5, NAND Flash 시장 동향 검색 (삼성전자 중심)                                                   
  samsung_hbm_trends = duckduckgo_search(query="2024 삼성전자 HBM 시장 동향 전망", max_results=3)                  
  print("삼성전자 HBM 시장 동향:", samsung_hbm_trends)                                                             
                                                                                                                   
  samsung_ddr5_trends = duckduckgo_search(query="2024 삼성전자 DDR5 시장 동향 전망", max_results=3)                
  print("삼성전자 DDR5 시장 동향:", samsung_ddr5_trends)                                                           
                                                                                                                   
  samsung_nand_trends = duckduckgo_search(query="2024 삼성전자 낸드플래시 시장 동향 전망", max_results=3)          
  print("삼성전자 NAND 시장 동향:", samsung_nand_trends)                                                           
                                                                                                                   
  # Step 1: HBM, DDR5, NAND Flash 시장 동향 검색 (SK하이닉스 중심)                                                 
  sk_hynix_hbm_trends = duckduckgo_search(query="2024 SK하이닉스 HBM 시장 동향 전망", max_results=3)               
  print("SK하이닉스 HBM 시장 동향:", sk_hynix_hbm_trends)                                                          
                                                                                                                   
  sk_hynix_ddr5_trends = duckduckgo_search(query="2024 SK하이닉스 DDR5 시장 동향 전망", max_results=3)             
  print("SK하이닉스 DDR5 시장 동향:", sk_hynix_ddr5_trends)                                                        
                                                                                                                   
  sk_hynix_nand_trends = duckduckgo_search(query="2024 SK하이닉스 낸드플래시 시장 동향 전망", max_results=3)       
  print("SK하이닉스 NAND 시장 동향:", sk_hynix_nand_trends)                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
삼성전자 HBM 시장 동향: Title: Hbsc "삼성전자 내년 상반기 Hbm4 격차 줄일 전망, 유럽·중동 투자자들 관심 집중"
Link: https://www.businesspost.co.kr/BP?command=article_view&num=414388
Snippet: HBM 시장 규모는 내년 전체 D램 시장의 42%에 달할 것으로 전망했으며, 2024 년부터 2026년까지 연평균 94% 
성장세를 이어갈 것으로 내다봤다. 또 범용 D램과 낸드플래시 등 기존 메모리반도체에서 장기 호황이 올 것이라고 
전망했다.

Title: 삼성전자 Hbm4 조기 출하 밸류체인 관련주 총정리: Ai 반도체 투자 핵심 포인트와 수혜주 분석
Link: 
https://jegwangkun.tistory.com/entry/삼성전자-HBM4-조기-출하-밸류체인-관련주-총정리-AI-반도체-투자-핵심-포인트와-수
혜주-분석
Snippet: 안녕하세요!오늘은 반도체 업계에서 가장 뜨거운 이슈 중 하나인 삼성전자의 HBM4 조기 출하와 그에 따른 
밸류체인 관련주 이야기를 나눠보려 합니다. 최근 AI 반도체 시장이 폭발적으로 성장하면서 고대역폭 메모리( HBM )의 
중요성이 더욱 커지고 있는데요, 삼성전자가 세계 최초로 HBM4를 조기 ...

Title: 삼성 HBM4: 반도체 시장의 판도 변화 - seo.goover.ai
Link: https://seo.goover.ai/report/202412/go-public-report-ko-ba64d0d0-92c8-45f7-b2aa-ad29c9bb1cb1-0-0.html
Snippet: 본 리포트는 삼성전자가 차세대 고대역폭메모리( HBM )인 HBM4의 개발을 통해 반도체 시장에서의 경쟁력을 
높이려는 전략을 분석합니다. HBM4는 데이터 전송 통로를 늘리고 전력 효율성을 높이기 위해 버퍼리스 HBM 기술을 
채택했습니다. 또한, 삼성전자는 TSMC와 협력하여 이러한 기술적 도전을 극복하고자 ...

삼성전자 DDR5 시장 동향: Title: 떨어진다던 DDR5 가격 돌연 반등 … 삼성전자 실적 회복 속도 붙나 | S...
Link: https://biz.newdaily.co.kr/site/data/html/2025/03/04/2025030400104.html
Snippet: Mar 4, 2025 · DDR 5 가격이 반등을 시작하면서 실적 저점이 될 것으로 예상됐던 올 1분기 메모리 반도체 시장 
분위기가 달라지고 있다. 특히 지난해 하반기 타격을 입었던 삼성전자 가 예상보다 빠르게 실적 회복에 나설 것이란 
기대감이 커지고 있다.

Title: 삼성전자 SK하이닉스 '호황이냐 불황이냐', 엇갈리는 하반기 메모리 ...
Link: https://www.businesspost.co.kr/BP?command=article_view&num=396042
Snippet: May 21, 2025 · [비즈니스포스트] 삼성전자 와 SK하이닉스가 주요 수요처들의 ‘선구매 효과’에 힘입어 D 램 
가격을 전격 인상하며, 반도체 호황 기대감을 키우고 있다.

Title: 글로벌 메모리 시장: DDR4·DDR5 가격 상승 사이클 (2025~2026 전망)
Link: https://moneyrecipe.blog/dram-market-trend-ai-demand/
Snippet: Nov 16, 2025 · 글로벌 메모리 시장이 AI와 클라우드 수요에 힘입어 구조적 상승 사이클에 진입했다. DDR4 공급 
부족은 대만 업체들에 기회를 주는 한편, 삼성전자 와 SK하이닉스는 DDR 5 ·HBM에서 강세가 전망된다.

삼성전자 NAND 시장 동향: Title: 삼성전자(A005930) | Consensus | 기업정보 | Company Guide
Link: https://comp.fnguide.com/SVO2/asp/SVD_Consensus.asp?gicode=A005930
Snippet: 전일자 보통주 수정주가 / 최근 결산 EPS(주당순이익) * EPS = 당기순이익 / 수정평균발행주식수 * 최근결산은 
2024/12 (연간) 기준 임. PER ... 시장대표업종||SUM(구성종목 시가총액)/SUM(구성종목 당기순이익) * 전일자 보통주 
시가총액 ...

Title: 낸드플래시 수요 살아났지만… 삼성전자·SK하이닉스, 공급량 확대에는 ‘신중’ - 조선비즈
Link: https://biz.chosun.com/it-science/ict/2024/04/16/AQKIFQQ57ZFSRDKP6JEO67YXSM/
Snippet: April 15, 2024 - 낸드플래시 수요 살아났지만 삼성전자·SK하이닉스, 공급량 확대에는 신중 삼성전자, 올 1분기 
낸드 생산량 30% 늘려 연말까지 낸드 가동률 상한선 50% 유지 美·日 경쟁사도 삼성 동향 ...

Title: "2024년 글로벌 반도체 매출 19% 증가··· 메모리 매출은 64% 급증" | CIO
Link: https://www.cio.com/article/3838081/2024년-글로벌-반도체-매출-19-증가···-메모리-매출은-64-급.html
Snippet: March 5, 2025 - 2024년 메모리 부문의 글로벌 매출은 2023년과 비교할 때 64% 급증할 전망 이며, 고대역폭 
메모리(HBM)가 매출 증가를 이끄는 역할을 했다. 로직 반도체 매출에서는 AI로 인한 GPU 수요가 ...

SK하이닉스 HBM 시장 동향: Title: SK하이닉스 2024 년 4분기 실적발표 컨퍼런스콜 전문 < 컨콜전문...
Link: https://www.thelec.kr/news/articleView.html?idxno=32511
Snippet: 올해는 AI 시장 중심의 견고한 HBM 수요를 바탕으로 해서 HBM 생산 극대화를 추진할 계획입니다.곽노정 
SK하이닉스 사장, “ HBM 3E 12단, 예정대로 간다”.

Title: SK하이닉스 , 2024 년 3분기 경영실적 발표 – SK hynix Newsroom
Link: https://news.skhynix.co.kr/3q-2024-business-results/
Snippet: 우선 D램을 보면, 회사는 기존 HBM 3에서 HBM 3E 8단 제품으로의 빠른 전환을 지속하고 있으며, 지난달 양산에 
들어간 HBM 3E 12단 제품의 공급도 예정대로 4분기에 시작할 계획이다.

Title: ‘36만닉스’ 현실 될까?...“ SK하이닉스 , 내년 HBM 시장 주도 전망 ”
Link: https://www.thepublic.kr/news/articleView.html?idxno=275885
Snippet: 이에 따라 2026년 SK하이닉스의 HBM 매출은 약 282억달러에 이를 것으로 추정됐다. 이는 HBM 3E 12단 가격 15% 
하락과 HBM 4 12단 점유율 70%를 가정한 보수적 시나리오로, 가격 하락 폭이 줄거나 공급량이 늘면 실적은 더 개선될 수 
있다는 분석이다. 김 연구원은 “ 시장 우려에도...

SK하이닉스 DDR5 시장 동향: Title: 삼성전자 Sk하이닉스 '호황이냐 불황이냐', 엇갈리는 하반기 메모리 시장 전망
Link: https://www.businesspost.co.kr/BP?command=article_view&num=396042
Snippet: D램 업체가 보유한 DDR5 재고는 2~3주에 불과하지만, 메모리 현물시장 최대 거래 업체인 에이데이타 (Adata)의 
현재 D램 재고 ( DDR5 비중 50%)는 20주 수준에 달하는 것으로 파악된다. 즉 실제 메모리 수요가 증가했다고 보기는 어려운
셈이다.

Title: 글로벌 메모리 시장: Ddr4·Ddr5 가격 상승 사이클 (2025~2026 전망)
Link: https://moneyrecipe.blog/dram-market-trend-ai-demand/
Snippet: 글로벌 메모리 시장이 AI와 클라우드 수요에 힘입어 구조적 상승 사이클에 진입했다. DDR4 공급 부족은 대만 
업체들에 기회를 주는 한편, 삼성전자와 SK 하이닉스는 DDR5·HBM에서 강세가 전망된다.

Title: 삼성전자 Sk 하이닉스 Ddr5 전략 

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # 다음 단계: 분석 및 한국어 요약                                                                                 
  analysis = """                                                                                                   
  [1] HBM 시장 동향 및 경쟁력                                                                                      
  - HBM 시장은 2024~2026년 연평균 약 94% 성장세가 예상될 정도로 폭발적인 확장세입니다.                             
  - 삼성전자는 HBM4 조기 출하 등 기술 경쟁력을 강화 중이지만, 핵심 고객인 엔비디아의 공식 납품 승인 및 신뢰도      
  면에서는 후발주자. 그러나 TSMC 등과의 협업을 통해 격차 해소에 집중.                                              
  - SK하이닉스는 HBM3/3E, HBM4까지 지속적으로 높은 점유율과 기술 리더십을 유지, 이미 엔비디아의 메인 파트너로      
  AI용 최고급 HBM을 대규모 납품하며 “AI 초기 승자”로 평가받음. 2026년에도 HBM 매출 고성장 전망.                    
                                                                                                                   
  [2] DDR5 및 NAND 시장                                                                                            
  - DDR5는 AI 데이터센터, PC 등에서 수요가 빠르게 증가하고 있으며, 2025년까지 DRAM 수익의 절반을 차지할            
  전망입니다. 삼성전자와 SK하이닉스 모두 강점.                                                                     
  - NAND Flash 시장은 2024년 이후 수요 반등세. 다만 가격/수급 안정화에 대해서는 보수적인 접근.                     
                                                                                                                   
  [3] 2024~2025년 실적 및 주가 전망                                                                                
  - 삼성전자는 최근 HBM(특히 HBM3E) 엔비디아 공급 승인 기대, DDR5·NAND 호황 등의 힘으로 빠른 실적 회복이 전망됨.   
  AI 메모리 선점 효과가 확대되면 시총 1,000조원, 주가 ‘10만전자’ 기대감도 보임.                                    
  - SK하이닉스는 AI HBM 시장에서 점유율과 성장성이 매우 높아, 높은 기술 신뢰성으로 실적·주가가 한동안 강세 지속이  
  전망됨. 다만 향후 공급 과잉/가격 하락 등은 리스크로 지적.                                                        
                                                                                                                   
  [4] 비교 및 종합 전망                                                                                            
  - HBM 등 AI 기반 메모리 시장이 양사 주가의 가장 중요한 차별점. 당장은 SK하이닉스가 ‘AI HBM의 독주자’로 삼성보다  
  실적 개선·주가 랠리에서 우위.                                                                                    
  - 삼성전자는 대규모 생산력/밸류체인 및 HBM4 등 신기술 조기 적용 시, 2025년 이후 HBM 경쟁 심화와 점유율 반전      
  가능.                                                                                                            
  - 결론적으로, 2024~2025년에는 SK하이닉스 상승세와 단기 강세, 삼성전자는 HBM 신제품 성과·엔비디아 수주 확대 시    
  중장기 모멘텀 확보가 예상됨.                                                                                     
  """                                                                                                              
  final_answer(analysis)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 
[1] HBM 시장 동향 및 경쟁력
- HBM 시장은 2024~2026년 연평균 약 94% 성장세가 예상될 정도로 폭발적인 확장세입니다. 
- 삼성전자는 HBM4 조기 출하 등 기술 경쟁력을 강화 중이지만, 핵심 고객인 엔비디아의 공식 납품 승인 및 신뢰도 
면에서는 후발주자. 그러나 TSMC 등과의 협업을 통해 격차 해소에 집중.
- SK하이닉스는 HBM3/3E, HBM4까지 지속적으로 높은 점유율과 기술 리더십을 유지, 이미 엔비디아의 메인 파트너로 AI용 
최고급 HBM을 대규모 납품하며 “AI 초기 승자”로 평가받음. 2026년에도 HBM 매출 고성장 전망.

[2] DDR5 및 NAND 시장
- DDR5는 AI 데이터센터, PC 등에서 수요가 빠르게 증가하고 있으며, 2025년까지 DRAM 수익의 절반을 차지할 전망입니다. 
삼성전자와 SK하이닉스 모두 강점.
- NAND Flash 시장은 2024년 이후 수요 반등세. 다만 가격/수급 안정화에 대해서는 보수적인 접근.

[3] 2024~2025년 실적 및 주가 전망
- 삼성전자는 최근 HBM(특히 HBM3E) 엔비디아 공급 승인 기대, DDR5·NAND 호황 등의 힘으로 빠른 실적 회복이 전망됨. AI 
메모리 선점 효과가 확대되면 시총 1,000조원, 주가 ‘10만전자’ 기대감도 보임.
- SK하이닉스는 AI HBM 시장에서 점유율과 성장성이 매우 높아, 높은 기술 신뢰성으로 실적·주가가 한동안 강세 지속이 
전망됨. 다만 향후 공급 과잉/가격 하락 등은 리스크로 지적.

[4] 비교 및 종합 전망
- HBM 등 AI 기반 메모리 시장이 양사 주가의 가장 중요한 차별점. 당장은 SK하이닉스가 ‘AI HBM의 독주자’로 삼성보다 
실적 개선·주가 랠리에서 우위.
- 삼성전자는 대규모 생산력/밸류체인 및 HBM4 등 신기술 조기 적용 시, 2025년 이후 HBM 경쟁 심화와 점유율 반전 가능.
- 결론적으로, 2024~2025년에는 SK하이닉스 상승세와 단기 강세, 삼성전자는 HBM 신제품 성과·엔비디아 수주 확대 시 
중장기 모멘텀 확보가 예상됨.

[Step 3: Duration 18.80 seconds| Input tokens: 14,798 | Output tokens: 1,473]


Final Report:

[1] HBM 시장 동향 및 경쟁력
- HBM 시장은 2024~2026년 연평균 약 94% 성장세가 예상될 정도로 폭발적인 확장세입니다. 
- 삼성전자는 HBM4 조기 출하 등 기술 경쟁력을 강화 중이지만, 핵심 고객인 엔비디아의 공식 납품 승인 및 신뢰도 면에서는 후발주자. 그러나 TSMC 등과의 협업을 통해 격차 해소에 집중.
- SK하이닉스는 HBM3/3E, HBM4까지 지속적으로 높은 점유율과 기술 리더십을 유지, 이미 엔비디아의 메인 파트너로 AI용 최고급 HBM을 대규모 납품하며 “AI 초기 승자”로 평가받음. 2026년에도 HBM 매출 고성장 전망.

[2] DDR5 및 NAND 시장
- DDR5는 AI 데이터센터, PC 등에서 수요가 빠르게 증가하고 있으며, 2025년까지 DRAM 수익의 절반을 차지할 전망입니다. 삼성전자와 SK하이닉스 모두 강점.
- NAND Flash 시장은 2024년 이후 수요 반등세. 다만 가격/수급 안정화에 대해서는 보수적인 접근.

[3] 2024~2025년 실적 및 주가 전망
- 삼성전자는 최근 HBM(특히 HBM3E) 엔비디아 공급 승인 기대, DDR5·NAND 호황 등의 힘으로 빠른 실적 회복이 전망됨. AI 메모리 선점 효과가 확대되면 시총 1,000조원, 주가 ‘10만전자’ 기대감도 보임.
- SK하이닉스는 AI HBM 시장에서 점유율과 성장성이 매우 높아, 높은 기술 신뢰성으로 실적·주가가 한동안 강세 지속이 전망됨. 다만 향후 공급 과잉/가격 하락 등은 리스크로 지적.

[4] 비교 및 종합 전망
- HBM 등 AI 기반 메모리 시장이 양사 주가의 가장 중요한 차별점. 당장은 SK하이닉스가 ‘AI HBM의 독주자’로 삼성보다 실적 개선·주가 랠리에서 우위.
- 삼성전자는 대규모 생산력/밸류체인 및 HBM4 등 신기술 조기 적용 시, 2025년 이후 HBM 경쟁 심화와 점유율 반전 가능.
- 결론적으로, 2024~2025년에는 SK하이닉스 상승세와 단기 강세, 삼성전자는 HBM 